<a href="https://colab.research.google.com/github/dvarim/gitlesson/blob/master/Lelchitskiy_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install faker psycopg2-binary

In [ ]:
import random
from datetime import date, timedelta
from faker import Faker
import psycopg2

In [ ]:
fake = Faker('ru_RU')

In [ ]:
conn = psycopg2.connect(
    host="158.160.76.203",
    database="course",
    user="course",
    password="course"
)
cur = conn.cursor()

In [ ]:
def random_date(start_year=1978, end_year=2025):
    year = random.randint(start_year, end_year)
    month = random.randint(1, 12)
    day = random.randint(1, 28)  # Избегаем ошибок с 29-31 числами
    return date(year, month, day)

In [ ]:
pip install faker psycopg2-binary

In [ ]:
!pip install psycopg2-binary
!pip install faker

from faker import Faker
import psycopg2
import random
import uuid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.1 MB/s eta 0:00:00


In [ ]:
# Конфигурация подключения к базе данных
DB_CONFIG = {
    'host': '158.160.76.203',
    'port': '5432',
    'database': 'course',
    'user': 'course',
    'password': 'course'
}

# Инициализация Faker
fake = Faker()

In [ ]:
def create_tables(conn):
    commands = (
        """
        CREATE TABLE IF NOT EXISTS ruslan_lelchitskiy.authors (
            author_id SERIAL PRIMARY KEY,
            first_name VARCHAR(100) NOT NULL,
            last_name VARCHAR(100) NOT NULL,
            birth_date DATE,
            country VARCHAR(100)
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS ruslan_lelchitskiy.books (
            book_id SERIAL PRIMARY KEY,
            title VARCHAR(255) NOT NULL,
            author_id INTEGER REFERENCES ruslan_lelchitskiy.authors(author_id),
            publication_year INTEGER,
            isbn VARCHAR(20) UNIQUE,
            genre VARCHAR(100),
            page_count INTEGER,
            available_copies INTEGER DEFAULT 1
        )
        """
    )
    try:
        cursor = conn.cursor()
        for command in commands:
            cursor.execute(command)
        conn.commit()
        print('Таблицы успешно созданы')
    except (Exception, psycopg2.DatabaseError) as error:
        print(f'Ошибка при создании таблиц: {error}')
    finally:
        cursor.close()

def generate_authors(num_authors):
    authors = []
    for _ in range(num_authors):
        birth_date = fake.date_of_birth(minimum_age=20, maximum_age=90)
        author = {
            'first_name': fake.first_name(),
            'last_name': fake.last_name(),
            'birth_date': birth_date,
            'country': fake.country()
        }
        authors.append(author)
    return authors

def generate_books(authors, books_per_author=2):
    books = []
    for author in authors:
        num_books = random.randint(2, 5)
        for _ in range(num_books):
            publication_year = fake.random_int(min=1900, max=2023)
            book = {
                'title': fake.sentence(nb_words=4).rstrip('.'),  # Убираем точку в конце
                'author_id': author['author_id'],
                'publication_year': publication_year,
                'isbn': fake.isbn13(),  # Генерируем уникальный ISBN
                'genre': random.choice(['Фантастика', 'Роман', 'Детектив', 'Боевик', 'Фэнтези']),
                'page_count': fake.random_int(min=50, max=1000),
                'available_copies': fake.random_int(min=1, max=10)
            }
            books.append(book)
    return books

def insert_data(conn, table_name, data):
    if not data:
        return
    columns = data[0].keys()
    query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"
    try:
        cursor = conn.cursor()
        for row in data:
            cursor.execute(query, list(row.values()))
        conn.commit()
        print(f'Данные успешно добавлены в таблицу: {table_name}')
    except (Exception, psycopg2.DatabaseError) as error:
        print(f'Ошибка при вставке данных в таблицу {table_name}: {error}')
    finally:
        cursor.close()

def main():
    conn = None
    try:
        conn = psycopg2.connect(**DB_CONFIG)

        # Создание таблиц
        create_tables(conn)

        # Генерация авторов
        num_authors = 1000
        authors = generate_authors(num_authors)
        insert_data(conn, 'ruslan_lelchitskiy.authors', authors)

        # Получение ID авторов из БД
        cursor = conn.cursor()
        cursor.execute('SELECT author_id FROM ruslan_lelchitskiy.authors')
        db_authors = [{'author_id': row[0]} for row in cursor.fetchall()]
        cursor.close()

        # Генерация книг
        books = generate_books(db_authors)
        insert_data(conn, 'ruslan_lelchitskiy.books', books)

    except (Exception, psycopg2.DatabaseError) as error:
        print(f'Ошибка: {error}')
    finally:
        if conn is not None:
            conn.close()
            print('Соединение с базой данных закрыто.')

if __name__ == '__main__':
    main()

Таблицы успешно созданы
Данные успешно добавлены в таблицу: ruslan_lelchitskiy.authors
Данные успешно добавлены в таблицу: ruslan_lelchitskiy.books
Соединение с базой данных закрыто.
